In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [3]:
torch.cuda.set_device(2)

In [4]:
# dls = get_3d_dls_aug(Meta.df_comb, sz=128, bs=32, grps=Meta.grps_stg1, meta=True)
dls = get_3d_dls_album(Meta.df_comb, path=path_jpg, sz=384, bs=1, grps=Meta.grps_stg1, meta=True)

In [5]:
xb,yb = dls.one_batch()
xb[0].shape, yb.shape

(torch.Size([1, 35, 3, 384, 384]), torch.Size([1, 35, 6]))

## Model

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, n_classes=6, embed_size=4096+1, LSTM_UNITS=4096+1, DO = 0.3):
        super(NeuralNet, self).__init__()
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x, pos):

#         x = torch.cat(x, axis=-1)
        h_embedding = torch.cat([x, pos], axis=-1)
        h_embadd = torch.cat((h_embedding, h_embedding), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
#         print([x.shape for x in [h_lstm1, h_lstm2, h_conc_linear1, h_conc_linear2, h_embadd]])
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [7]:
import pretrainedmodels

In [8]:
# class ReshapeBodyHook():
#     def __init__(self, body):
#         super().__init__()
#         self.pre_reg = body.register_forward_pre_hook(self.pre_hook)
#         self.reg = body.register_forward_hook(self.forward_hook)
#         self.shape = None

#     def deregister(self):
#         self.reg.remove()
#         self.pre_reg.remove()

#     def pre_hook(self, module, input):
#         print('skfjsdklfj')
#         x = input[0]
#         self.shape = x.shape
#         return (x.view(-1, *x.shape[2:]),)

#     def forward_hook(self, module, input, x):
#         print('hgfjdkhgd')
#         return x.view(*self.shape[:2], *x.shape[1:])

In [9]:
class CombMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.body = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
        self.body.avg_pool = AdaptiveConcatPool2d()
        self.body.last_linear = Flatten()
        
#         self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.body)
        
        self.head = NeuralNet()
        
    def forward(self, x):
        x,pos = x
        x = self.body(x)
        return self.head(x,pos)

In [10]:
m = CombMod().cuda()

In [12]:
m.body.load_state_dict(torch.load(Path(f'models/runs/04_xse_resnext_512_appian_alb-1.pth'), map_location='cpu')['model'], strict=False)
# m.head.load_state_dict(torch.load(Path(f'models/runs/train3d_adj_feat_lstm_2ndplace_meta-1.pth'), map_location='cpu')['model'])

_IncompatibleKeys(missing_keys=[], unexpected_keys=['last_linear.weight', 'last_linear.bias'])

In [13]:
learn = get_learner(dls, m, get_loss())

In [14]:
learn.add_cb(DePadLoss())

In [15]:
name = 'train3d_baseline_lstm_2nd_full_meta'

In [34]:
# learn.summary()

## Training

In [35]:
# learn.lr_find()

In [36]:
xb,yb = dls.one_batch()

In [37]:
xb[0].shape

torch.Size([2, 60, 3, 384, 384])

In [16]:
do_fit(learn, 4, 1e-3)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time


RuntimeError: CUDA out of memory. Tried to allocate 258.00 MiB (GPU 2; 23.65 GiB total capacity; 22.54 GiB already allocated; 14.00 MiB free; 318.26 MiB cached)

## Test

In [13]:
learn.load(f'runs/{name}-2-no256', strict=True)
sub_fn = f'subm/{name}'

In [14]:
dls = get_3d_dls_aug(Meta.df_tst, sz=384, path=path_tst_jpg, bs=8, test=True)

In [15]:
learn.dls = dls

In [16]:
# learn = get_learner(dls, learn.model)
# hook.deregister()
# hook = ReshapeBodyHook(learn.model[0])
# learn.add_cb(DePadLoss())

In [17]:
preds,targs = learn.get_preds()

In [18]:
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(1.2397e-13), tensor(0.9998))

In [19]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [20]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 10.3MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [29]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14325387,
 'totalBytes': 27277209,
 'date': '2020-01-29T19:15:10.047Z',
 'description': 'train3d_baseline_lstm_2nd',
 'errorDescription': None,
 'fileName': 'train3d_baseline_lstm_2nd.csv',
 'publicScore': '0.85209',
 'privateScore': '0.05333',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14325387/14325387.raw'}